<a href="https://colab.research.google.com/github/ishaqmarashy/red-light-traffic-violation-classification/blob/main/decision%20tree%20classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

csv_url = "./data/Traffic_Violations.csv"
##csv_url = "https://data.montgomerycountymd.gov/api/views/4mse-ku6q/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(csv_url)

In [12]:
list(df)

['SeqID',
 'Date Of Stop',
 'Time Of Stop',
 'Agency',
 'SubAgency',
 'Description',
 'Location',
 'Latitude',
 'Longitude',
 'Accident',
 'Belts',
 'Personal Injury',
 'Property Damage',
 'Fatal',
 'Commercial License',
 'HAZMAT',
 'Commercial Vehicle',
 'Alcohol',
 'Work Zone',
 'Search Conducted',
 'Search Disposition',
 'Search Outcome',
 'Search Reason',
 'Search Reason For Stop',
 'Search Type',
 'Search Arrest Reason',
 'State',
 'VehicleType',
 'Year',
 'Make',
 'Model',
 'Color',
 'Violation Type',
 'Charge',
 'Article',
 'Contributed To Accident',
 'Race',
 'Gender',
 'Driver City',
 'Driver State',
 'DL State',
 'Arrest Type',
 'Geolocation']

In [13]:
# Removed columns that do not contribute to outcome or are incomplete
# Some columns I removed here like longitude may make a nice visual for later
columns_to_drop = [
        'SeqID',
        'HAZMAT',
        'Search Conducted',
        'Search Disposition',
        'Search Outcome',
        'Search Type',
        'Article',
        'Contributed To Accident',
        'Arrest Type',
        'Search Reason',
        'Search Reason For Stop',
        'Charge',
        'Search Arrest Reason']


# Drop the specified columns
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
df.isna().sum()

Date Of Stop              0
Time Of Stop              0
Agency                    0
SubAgency                 0
Description              10
Location                  4
Latitude                  0
Longitude                 0
Accident                  0
Belts                     0
Personal Injury           0
Property Damage           0
Fatal                     0
Commercial License        0
Commercial Vehicle        0
Alcohol                   0
Work Zone                 0
State                    59
VehicleType               0
Year                  10375
Make                     68
Model                   213
Color                 21654
Violation Type            0
Race                      0
Gender                    0
Driver City             474
Driver State             11
DL State                929
Geolocation               0
dtype: int64

In [14]:
df.dropna(subset=['Description'], inplace=True)

In [15]:
def replace_description_with_most_frequent_value(df, keywords, field):
    filtered_rows=df
    for keyword in keywords:
        filtered_rows = filtered_rows[filtered_rows[field].str.contains(keyword, case=False)]

    if not filtered_rows.empty:
        most_frequent_value = filtered_rows[field].mode().iloc[0]
        df.loc[filtered_rows.index, field] = most_frequent_value

In [16]:
df['Fatal'].value_counts()

Fatal
No     1879404
Yes        498
Name: count, dtype: int64

In [17]:
soonest_date = df['Date Of Stop'].min()
oldest_date = df['Date Of Stop'].max()

print("Soonest Date:", soonest_date)
print("Oldest Date:", oldest_date)

Soonest Date: 01/01/2012
Oldest Date: 12/31/2022


In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishaq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
df['Description'].value_counts()

Description
DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC CONTROL DEVICE INSTRUCTIONS                     156937
FAILURE TO DISPLAY REGISTRATION CARD UPON DEMAND BY POLICE OFFICER                              84442
DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGISTRATION                                          55131
FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DISPLAY LICENSE TO UNIFORMED POLICE ON DEMAND       49804
DISPLAYING EXPIRED REGISTRATION PLATE ISSUED BY ANY STATE                                       47429
                                                                                                ...  
DRIVING VEH W/ TV-TYPE RECEIVING EQUIPTHAT IS ON AND VISIBLE TO THE DRIVER)ON HWY                   1
DRIVING VEHICLE IN EXCESS OF REASONABLE AND PRUDENT SPEED ON HIGHWAY 66/40 ZONE                     1
PARKING VEH. LEFT WHEELS TO CURB                                                                    1
DRIVE, ATTEMPT TO DRIVE VEH. WHILE IMPAIRED BY DRUG OR DRUG AND ALCOHO

In [21]:
import pandas as pd
from itertools import combinations
from collections import Counter


df['Tokens'] = df['Description'].str.split()

def common_words(desc1, desc2, min_common_words):
    common = set(desc1) & set(desc2)
    if len(common) >= min_common_words:
        return ' '.join(common)
    else:
        return ' '.join(desc1)

combinations = list(combinations(df['Tokens'], 2))
min_common_words = 3

for desc1, desc2 in combinations:
    common = common_words(desc1, desc2, min_common_words)
    if common:
        df['Description'] = df['Description'].str.replace(f"{' '.join(desc1)}|{' '.join(desc2)}", common)

df.drop(columns=['Tokens'], inplace=True)

print(df)

In [ ]:
df['Description'].value_counts()

Description
A description with some words here.    1
A different description.               1
A unique text.                         1
Some words in this description.        1
Name: count, dtype: int64